In [1]:
import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine, func, extract
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [4]:
# Reflect an existing database into a new model:
Base = automap_base()

# Reflect the tables:
Base.prepare(engine, reflect=True)

In [5]:
# Save references to each table
Measurement = Base.classes.measurement

In [6]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [7]:
# Challenge steps:
# 1. Determine the date range covered in the analysis
# 2. Extract JUN data across all years for 'prcp' & 'tobs', then describe()
# 3. Extract DEC data across all years for 'prcp' & 'tobs', then describe()
# 4. Bar graph JUN & DEC to visually compare

In [8]:
# 1A. Find the first date in the range:
DRange = session.query(Measurement.date).\
        order_by((Measurement.date).desc()).all()
FirstDate = DRange[-1]
FirstDate

('2010-01-01')

In [10]:
# 1B. Find the last date in the range:
LastDate = DRange[0]
LastDate

('2017-08-23')

In [23]:
# 2A. Extract the precipitation & temperature observations for JUN across all years:

from sqlalchemy import extract  

JUN_extract = session.query(Measurement.date, Measurement.prcp, Measurement.tobs).\
filter(extract('month', Measurement.date)==6).all()

# Save the query results as a Pandas DataFrame and set the index to the date column:

JUN_data = pd.DataFrame(JUN_extract, columns=['date', 'precipitation', 'temperatures'])
JUN_data.set_index(JUN_data['date'], inplace=True)
JUN_data.sort_index(ascending=True)
print(JUN_data.to_string(index=False))

       date  precipitation  temperatures
 2010-06-01           0.00          78.0
 2010-06-02           0.01          76.0
 2010-06-03           0.00          78.0
 2010-06-04           0.00          76.0
 2010-06-05           0.00          77.0
 2010-06-06           0.00          78.0
 2010-06-07           0.00          77.0
 2010-06-08           0.00          78.0
 2010-06-09           0.00          78.0
 2010-06-10           0.00          79.0
 2010-06-11           0.00          78.0
 2010-06-12           0.00          78.0
 2010-06-13           0.00          78.0
 2010-06-14           0.00          77.0
 2010-06-15           0.00          78.0
 2010-06-16           0.00          78.0
 2010-06-17           0.00          77.0
 2010-06-18           0.00          77.0
 2010-06-19           0.00          82.0
 2010-06-20           0.00          78.0
 2010-06-21           0.00          78.0
 2010-06-22           0.00          78.0
 2010-06-23           0.00          78.0
 2010-06-24     

In [22]:
# 3A. Extract the precipitation & temperature observations for DEC across all years:

DEC_extract = session.query(Measurement.date, Measurement.prcp, Measurement.tobs).\
filter(extract('month', Measurement.date)==12).all()

# Save the query results as a Pandas DataFrame and set the index to the date column:

DEC_data = pd.DataFrame(DEC_extract, columns=['date', 'precipitation', 'temperatures'])
DEC_data.set_index(DEC_data['date'], inplace=True)
DEC_data.sort_index(ascending=True)
print(DEC_data.to_string(index=False))

       date  precipitation  temperatures
 2010-12-01           0.04          76.0
 2010-12-03           0.00          74.0
 2010-12-04           0.00          74.0
 2010-12-06           0.00          64.0
 2010-12-07           0.00          64.0
 2010-12-08           0.00          67.0
 2010-12-09           0.00          77.0
 2010-12-10           1.58          66.0
 2010-12-11           0.77          69.0
 2010-12-12           0.00          68.0
 2010-12-13           0.00          68.0
 2010-12-14           0.00          71.0
 2010-12-15           0.00          74.0
 2010-12-16           0.00          67.0
 2010-12-17           0.00          66.0
 2010-12-18           0.00          69.0
 2010-12-19           1.40          71.0
 2010-12-23           0.00          70.0
 2010-12-24           0.00          70.0
 2010-12-26            NaN          74.0
 2010-12-27           0.13          74.0
 2010-12-28           0.55          71.0
 2010-12-29           0.06          75.0
 2010-12-30     

In [14]:
# 2B. Determine key statistical data about the month of June across all years:
JUN_data.describe()

,precipitation,temperatures
count,1574.000000,1700.000000
mean,0.136360,74.944118
std,0.335731,3.257417
min,0.000000,64.000000
25%,0.000000,73.000000
50%,0.020000,75.000000
75%,0.120000,77.000000
max,4.430000,85.000000


In [15]:
# 3B. Determine key statistical data about the month of June across all years:
DEC_data.describe()

,precipitation,temperatures
count,1405.000000,1517.000000
mean,0.216819,71.041529
std,0.541399,3.745920
min,0.000000,56.000000
25%,0.000000,69.000000
50%,0.030000,71.000000
75%,0.150000,74.000000
max,6.420000,83.000000


In [29]:
# Import the API key:
from config import g_key
import gmaps

In [27]:
Station = Base.classes.station
Station_extract = session.query(Station.station, Station.name, Station.latitude, Station.longitude, Station.elevation).all()

Station_data = pd.DataFrame(Station_extract, columns=['station', 'name', 'latitude', 'longitude', 'elevation'])
Station_data.set_index(Station_data['station'], inplace=True)
print(Station_data.to_string(index=False))

     station                                    name  latitude  longitude  elevation
 USC00519397                    WAIKIKI 717.2, HI US  21.27160 -157.81680        3.0
 USC00513117                    KANEOHE 838.1, HI US  21.42340 -157.80150       14.6
 USC00514830  KUALOA RANCH HEADQUARTERS 886.9, HI US  21.52130 -157.83740        7.0
 USC00517948                       PEARL CITY, HI US  21.39340 -157.97510       11.9
 USC00518838              UPPER WAHIAWA 874.3, HI US  21.49920 -158.01110      306.6
 USC00519523      WAIMANALO EXPERIMENTAL FARM, HI US  21.33556 -157.71139       19.5
 USC00519281                     WAIHEE 837.5, HI US  21.45167 -157.84889       32.9
 USC00511918       HONOLULU OBSERVATORY 702.2, HI US  21.31520 -157.99920        0.9
 USC00516128            MANOA LYON ARBO 785.2, HI US  21.33310 -157.80250      152.4


In [49]:
# Create information pop up box for hotels:
info_box_template = """
<dl>
<dt>Station ID</dt><dd>{station}</dd>
<dt>Name</dt><dd>{name}</dd>
<dt>Elevation</dt><dd>{elevation}</dd>
</dl>
"""

# Store the DataFrame Row.
station_info = [info_box_template.format(**row) for index, row in Station_data.iterrows()]

In [50]:
# Add a heatmap of preferred locations for the vacation spots
gmaps.configure(api_key = g_key)
locations = Station_data[["latitude", "longitude"]]

fig = gmaps.figure(center=(21.48, -157.85), zoom_level=10.0)

marker_layer = gmaps.marker_layer(locations, 
                                  info_box_content=station_info)
                                  
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))